In [2]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox

# Function to load accounts from a file
def load_accounts(filename):
    try:
        with open(filename, "r") as file:
            accounts_data = eval(file.read())
            return accounts_data
    except FileNotFoundError:
        return None

# Function to save accounts to a file
def save_accounts(accounts_data, filename):
    with open(filename, "w") as file:
        file.write(str(accounts_data))

# Load accounts from a file
filename = "accounts.txt"
accounts_data = load_accounts(filename)
if accounts_data:
    accounts = accounts_data

# Define a function to update the balance
def update_balance(amount):
    global balance
    balance += amount
    balance_label.config(text=f"Current Balance: {balance}$")
    # Update the balance in the accounts dictionary
    accounts[account_name]["balance"] = balance
    # Save the updated accounts to the file
    save_accounts(accounts, filename)

# Define a function to withdraw money
def withdraw_money():
    global balance
    amount = int(amount_entry.get())
    if amount <= balance:
        update_balance(-amount)
        messagebox.showinfo("Success", f"Withdrawal of {amount}$ successful")
    else:
        messagebox.showerror("Error", "Insufficient balance")

# Define a function to deposit money
def deposit_money():
    global balance
    amount = int(amount_entry.get())
    update_balance(amount)
    messagebox.showinfo("Success", f"Deposit of {amount}$ successful")

# Define a function to check the password and select the account
def check_password():
    global balance, password, account_name
    account_name = account_entry.get()
    if account_name in accounts:
        password = accounts[account_name]["password"]
        if password_entry.get() == password:
            balance = accounts[account_name]["balance"]
            amount_entry.config(state="normal")
            withdraw_button.config(state="normal")
            deposit_button.config(state="normal")
            # Add face recognition
            cap = cv2.VideoCapture(0)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in faces:
                    roi_gray = gray[y:y + h, x:x + w]
                    roi_color = frame[y:y + h, x:x + w]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Face Recognized", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    # Check if face is recognized and exit the loop
                    if len(faces) > 0:
                        cap.release()
                        cv2.destroyAllWindows()
                        balance_label.config(text=f"Current Balance: {balance}$")
                        return
                cv2.imshow('frame', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()
        else:
            messagebox.showerror("Error", "Password incorrect")
    else:
        messagebox.showerror("Error", "Account not found")

# Define a function to exit the program
def exit_program():
    if messagebox.askokcancel("Exit", "Are you sure you want to exit?"):
        root.destroy()

# Create the main window
root = tk.Tk()
root.title("My Program")

# Create a canvas with gradient background
canvas = tk.Canvas(root, width=1600, height=1200)
canvas.pack()

# Define the gradient colors
start_color = "#eeaeca"
end_color = "#0D47A1"

# Create gradient rectangles
for i in range(1200):
    r = int(i * (int(end_color[1:3], 16) - int(start_color[1:3], 16)) / 1200 + int(start_color[1:3], 16))
    g = int(i * (int(end_color[3:5], 16) - int(start_color[3:5], 16)) / 1200 + int(start_color[3:5], 16))
    b = int(i * (int(end_color[5:7], 16) - int(start_color[5:7], 16)) / 1200 + int(start_color[5:7], 16))
    color = f"#{r:02x}{g:02x}{b:02x}"
    canvas.create_rectangle(0, i, 1600, i + 1, fill=color, width=0)


# Create the widgets
welcome_label = tk.Label(root, text="Welcome To The ATM Machine", font=("Arial", 50), bg="#96f514")
account_label = tk.Label(root, text="Enter account number:", bg="yellow",font=("Arial",20))
account_entry = tk.Entry(root, font=("Arial", 16,))
password_label = tk.Label(root, text="Enter password:", bg="yellow",font=("Arial",20))
password_entry = tk.Entry(root, show="*", font=("Arial", 16))
password_button = tk.Button(root, text="Enter", font=("Arial", 16), command=check_password)
balance_label = tk.Label(root, text="", font=("Arial", 20), bg=end_color)
amount_label = tk.Label(root, text="Enter amount:", bg="yellow",font=("Arial",20))
amount_entry = tk.Entry(root, font=("Arial", 16), state="disabled")
withdraw_button = tk.Button(root, text="Withdraw", font=("Arial", 16), command=withdraw_money, state="disabled")
deposit_button = tk.Button(root, text="Deposit", font=("Arial", 16), command=deposit_money, state="disabled")
#exit_button = tk.Button(root, text="Exit", font=("Arial", 16), bg=start_color, command=exit_program)
#author = tk.Label(root, text="copyright@Vaibhav", font=("Arial", 10), bg=start_color)
# Create the circular exit button
button_radius = 40
button_center_x = 800
button_center_y = 620
button_outline_color = "yellow"
button_fill_color = "red"
canvas.create_oval(
    button_center_x - button_radius,
    button_center_y - button_radius,
    button_center_x + button_radius,
    button_center_y + button_radius,
    outline=button_outline_color,
    fill=button_fill_color,
    width=2
)

# Create a text label for the exit button
button_label = tk.Label(root, text="Exit", font=("Arial", 16))
button_label.place(x=button_center_x, y=button_center_y, anchor="center")

# Bind the button label and canvas to the exit function
button_label.bind("<Button-1>", lambda event: exit_program())
canvas.bind("<Button-1>", lambda event: exit_program())


# Add the widgets to the canvas
canvas.create_window(800, 35, window=welcome_label)
canvas.create_window(800, 150, window=account_label)
canvas.create_window(800, 200, window=account_entry)
canvas.create_window(800, 280, window=password_label)
canvas.create_window(800, 330, window=password_entry)
canvas.create_window(800, 380, window=password_button)
canvas.create_window(800, 430, window=balance_label)
canvas.create_window(800, 480, window=amount_label)
canvas.create_window(800, 530, window=amount_entry)
canvas.create_window(800 - 200, 530 , window=withdraw_button)
canvas.create_window(800 + 200, 530, window=deposit_button)
#canvas.create_window(800, 580, window=exit_button)
#canvas.create_window(1500, 1100+20, window=author)

# Run the Tkinter event loop
root.mainloop()
